# AWS Virtual Private Cloud

A VPC is a logically isolated networking environment for your AWS resources. In this notebook we'll create a VPC, and partition it into public and private subnets, showing how we can use subnets as security containers or boundaries.

## Creating a VPC

In [ ]:
import boto3

client = boto3.client('ec2')
vpc_response = client.create_vpc(
    CidrBlock='10.0.0.0/16'
)

vpcId = vpc_response['Vpc']['VpcId']

print vpc_response

## Subnets

Now that we have a VPC, we can create subnets. In contrast to a VPC, which spans all the availability zones in
a region, a subnet is specific to an availability zone.

So the first thing to figure out is what are the availability zones in the region.

In [ ]:
response = client.describe_availability_zones()
availability_zones = response['AvailabilityZones']

for az in availability_zones:
    print az

One we know the zones, we can figure out if we can create a subnet in the zone by doing a dry run.

TO DO: try this code in us-west-1, which has 3 AZs with only two of those supporting VPC subnet creation. I can create subnets in all AZs in us-east-1, so I just flip DryRun to false in the code below.

In [ ]:
# Example code to create a subnet in each AZ
subnet_no = -1
cidr_base = '10.0.'
for az in availability_zones:
    subnet_no += 1
    zone_name = az['ZoneName']
    
    print 'create subnet in zone ', zone_name, ' for subnet ', subnet_no
    
    try:
        response = client.create_subnet(
            DryRun=True,
            VpcId = vpcId,
            CidrBlock = cidr_base + str(subnet_no) + '.0/24',
            AvailabilityZone = zone_name
        )

        print response
        
        
    except Exception, Argument:
        print Exception
        print Argument
        
        


In [ ]:
# For the rest of this work book we'll create two subnets in two AZs
availability_zones = availability_zones[0:2]
subnet_no = -1
cidr_base = '10.0.'
for i in range(2):
    subnet_no += 1
    zone_name = availability_zones[i]['ZoneName']
    
    print 'create subnet in zone ', zone_name, ' for subnet ', subnet_no
    
    response = client.create_subnet(
        VpcId = vpcId,
        CidrBlock = cidr_base + str(subnet_no) + '.0/24',
        AvailabilityZone = zone_name
    )
    

    subnet_no += 1    
    print 'create subnet in zone ', zone_name, ' for subnet ', subnet_no
    
    response = client.create_subnet(
        VpcId = vpcId,
        CidrBlock = cidr_base + str(subnet_no) + '.0/24',
        AvailabilityZone = zone_name
    )    
   
    


## Clean Up

In [ ]:
# TODO - delete all the subnets in the VPC before deleting the VPC
response = client.describe_subnets(
    Filters=[
        {
            'Name': 'vpc-id',
            'Values': [
                vpcId,
            ]
        },
    ]
)

subnets = response['Subnets']

for sn in subnets:
    response = client.delete_subnet(
        SubnetId=sn['SubnetId']
    )
    
    print response

# Delete the VPC
response = client.delete_vpc(
    VpcId = vpcId
)

print response